In [1]:
pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 82.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json, os
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [4]:
# === Config ===
DATA_FILE = "/content/drive/MyDrive/AI/faculty_data.json"
EMBEDDING_OUTPUT_PATH = "/content/drive/MyDrive/AI/embeddings/faculty_faiss_index.bin"
METADATA_OUTPUT_PATH = "/content/drive/MyDrive/AI/embeddings/faculty_metadata.json"

In [5]:
# === Load Data ===
with open(DATA_FILE, "r") as f:
    raw_data = json.load(f)

# === Load Embedding Model ===
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Process Data ===
metadata = []
texts = []

for prof in raw_data:
    try:
        content = f"""
Name: {prof['name']}
Designation: {prof['designation']}
Department: {prof['department']}
Email: {prof['email']}
Courses Taught: {', '.join(prof.get('courses_taught', []))}
Recent Publications: {', '.join(prof.get('recent_publications', []))}
Ongoing Projects: {prof.get('ongoing_projects', '')}
Research Interests: {prof.get('research_interests', '')}
Research Keywords: {', '.join(prof.get('research_keywords', []))}
""".strip()

        texts.append(content)
        metadata.append({
            "name": prof["name"],
            "designation": prof["designation"],
            "department": prof["department"],
            "email": prof["email"],
            "courses_taught": prof.get("courses_taught", []),
            "recent_publications": prof.get("recent_publications", []),
            "ongoing_projects": prof.get("ongoing_projects", ""),
            "research_interests": prof.get("research_interests", ""),
            "research_keywords": prof.get("research_keywords", []),
            "content": content
        })
    except Exception as e:
        print(f"❌ Error processing entry: {prof.get('name', 'Unknown')} - {e}")

# === Encode & Save ===
embeddings = bert_model.encode(texts, convert_to_numpy=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

faiss.write_index(index, EMBEDDING_OUTPUT_PATH)
with open(METADATA_OUTPUT_PATH, "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Data converted and index saved.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Data converted and index saved.


In [6]:
# === SECTION 2: Model Loading and Initialization ===

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# === Paths ===
drive_folder = "/content/drive/MyDrive/AI"
metadata_path = os.path.join(drive_folder, "embeddings/faculty_metadata.json")
index_path = os.path.join(drive_folder, "embeddings/faculty_faiss_index.bin")

# === Load Metadata ===
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# === Load FAISS Index ===
faiss_index = faiss.read_index(index_path)

# === Load Sentence-BERT model for query embedding ===
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Load Phi model for generation ===
use_phi15 = False  # Set to True to use smaller, faster phi-1_5
phi_model_id = "microsoft/phi-1_5" if use_phi15 else "microsoft/phi-2"
phi_tokenizer = AutoTokenizer.from_pretrained(phi_model_id)
phi_model = AutoModelForCausalLM.from_pretrained(
    phi_model_id,
    device_map="auto",
    torch_dtype="auto"
)

# === Warmup Phi model to prevent cold-start lag ===
phi_model.generate(
    **phi_tokenizer("warmup", return_tensors="pt").to(phi_model.device),
    max_new_tokens=1
)

print("✅ Models and Index Loaded Successfully")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ Models and Index Loaded Successfully


In [8]:
# === Memory Management ===
MAX_MEMORY = 10

def update_memory(memory, query, answer, max_length=5):
    memory.append({
        "user": query,
        "assistant": answer
    })
    return memory[-max_length:]  # Keep only the last 5


# def build_memory_prompt(memory, context, query):
#     memory_str = ""
#     if memory:
#         print("\n🧠 Memory Contents:")
#         for idx, item in enumerate(memory[-5:]):
#             print(f"Q{idx+1}: {item['user']}")
#             print(f"A{idx+1}: {item['assistant']}\n")
#             memory_str = "\n".join([f"Q: {m['user']}\nA: {m['assistant']}" for m in memory[-5:]])

#     prompt = f"""You are a helpful academic assistant. Use the previous conversation and context to answer accurately.
# Only respond based on the faculty data. Do not make up facts.

# Previous Conversation:
# {memory_str}

# Context from Faculty Data:
# {context}

# Question: {query}

# Answer:"""

#     return prompt


In [9]:
def build_memory_prompt(memory, context, query, include_memory=True, max_turns=5, debug_print=True):
    """
    Build the final prompt for Phi-2.

    include_memory = False  -> do NOT feed prior conversation to the model
                              (use this when a professor name is already resolved)
    max_turns: how many recent QA pairs to include if include_memory=True
    debug_print: print memory contents to console regardless (for observability)
    """
    # --- Prepare memory block (for prompt and/or debug) ---
    recent = memory[-max_turns:] if memory else []
    if debug_print and recent:
        print("\n🧠 Memory (latest first shown last):")
        for i, m in enumerate(recent, start=1):
            print(f"Q{i}: {m.get('user','')}")
            print(f"A{i}: {m.get('assistant','')}\n")

    memory_str = ""
    if include_memory and recent:
        memory_str = "\n".join([f"Q: {m.get('user','')}\nA: {m.get('assistant','')}" for m in recent])
    elif debug_print and recent:
        print("⚠️ Memory is NOT included in the prompt for this turn (name-locked or disambiguated query).")

    # --- Build prompt ---
    prompt = f"""You are a helpful academic assistant. Use the context to answer accurately.
Only respond based on the faculty data. Do not make up facts.

Previous Conversation:
{memory_str}

Context from Faculty Data:
{context}

Question: {query}

Answer:"""

    return prompt


In [10]:
import gradio as gr

def find_prof_by_name(query, metadata):
    """Return metadata entry if a professor's name is mentioned in the query."""
    query_lower = query.lower()
    for prof in metadata:
        name_parts = prof["name"].lower().split()
        if all(part in query_lower for part in name_parts):  # full match
            return prof
        elif any(part in query_lower for part in name_parts):  # partial match
            return prof
    return None

def is_ambiguous_query(query):
    pronouns = ["him", "her", "he", "she", "his","they", "them"]
    return any(p in query.lower().split() for p in pronouns)

def resolve_query_via_memory(query, memory, metadata):
    """
    If the query is ambiguous (pronouns) and we have memory,
    scan memory from newest to oldest and return the most recently
    mentioned professor (full metadata dict). Otherwise return None.
    """
    if not is_ambiguous_query(query) or not memory:
        return None

    # Scan newest → oldest; look for any professor name mentioned
    for turn in reversed(memory):
        hay = f"{turn.get('user','')} {turn.get('assistant','')}".lower()
        for prof in metadata:
            if prof["name"].lower() in hay:
                print(f"🔁 Ambiguity resolved via memory → using: {prof['name']}")
                return prof  # return the actual metadata chunk

    # Nothing found in memory
    return None



# === Util: Reranking Function ===
def rerank_results(query, chunks):
    priority = []
    query_lower = query.lower()

    for c in chunks:
        score = 0
        dept = c.get("department", "").lower()
        r_keywords = [kw.lower() for kw in c.get("research_keywords", [])]
        r_interests = c.get("research_interests", "").lower()
        projects = c.get("ongoing_projects", "").lower()
        courses = [c_name.lower() for c_name in c.get("courses_taught", []) if isinstance(c_name, str)]

        if any(word in query_lower for word in dept.split()):
            score += 1
        if any(kw in query_lower for kw in r_keywords):
            score += 1
        if r_interests and r_interests in query_lower:
            score += 1
        if any(cname in query_lower for cname in courses):
            score += 0.5
        if any(word in query_lower for word in projects.split()):
            score += 0.5

        priority.append((score, c))

    priority.sort(reverse=True, key=lambda x: x[0])

    print("🔍 Top reranked candidates:")
    for score, chunk in priority[:5]:
        print(f"✔️ {chunk['name']} | Score: {score:.1f} | Dept: {chunk['department']}")

    return [x[1] for x in priority if x[0] > 0]

# === Util: Structured Logic Fallback ===
def structured_query_response(query):
    query_lower = query.lower()
    results = []

    for prof in metadata:
        name = prof.get("name", "").lower()
        dept = prof.get("department", "").lower()
        research = prof.get("research_interests", "").lower()
        keywords = " ".join(prof.get("research_keywords", [])).lower()

        if any(name_part in query_lower for name_part in name.split()):
            results.append(
                f"👤 {prof['name']} ({prof['designation']}, {prof['department']})\n📘 Research Interests: {prof['research_interests']}"
            )

    return "\n\n".join(results) if results else None


def truncate(text, max_words=400):
    return " ".join(text.split()[:max_words])

def build_context(top_chunks):
    return "\n\n".join(
        [f"{m['name']} ({m['designation']}, {m['department']}):\n{m['content']}" for m in top_chunks]
    )

# === Pipeline 1: Faculty Research Pipeline ===
def faculty_pipeline(query, history=[], memory=[]):
    print(f"\n🧠 Incoming Faculty Query: {query}")

    # 1) Exact name in current query?
    matched_prof = find_prof_by_name(query, metadata)

    # 2) If not, and query is ambiguous, try resolve from memory
    if not matched_prof and is_ambiguous_query(query):
        matched_prof = resolve_query_via_memory(query, memory, metadata)

    if matched_prof:
        print(f"🎯 Using name-based retrieval: {matched_prof['name']} (skip FAISS)")
        top_chunks = [matched_prof]
        name_path = True
    else:
        # 3) Fall back to FAISS only when no direct name available
        q_embed = bert_model.encode([query], convert_to_numpy=True)
        D, I = faiss_index.search(q_embed, k=10)
        top_chunks = [metadata[i] for i in I[0]]
        name_path = False

    # Use top N chunks
    # top_chunks = top_chunks[:5]

    # Print chunks for logging/debug

    print("\n📥 Retrieved Context Chunks:\n")
    for idx, chunk in enumerate(top_chunks):
        print(f"--- Chunk {idx+1} ---")
        print(f"Name: {chunk['name']}")
        print(f"Department: {chunk['department']}")
        print(f"Email: {chunk['email']}")
        print(f"Courses: {chunk.get('courses_taught', [])}")
        print(f"Content:\n{chunk['content']}\n")

        print("🧭 Using FAISS fallback")
    # === Log Retrieved Chunks ===
    print("\n📥 Retrieved Context Chunks:")
    for idx, chunk in enumerate(top_chunks):
        print(f"\n--- Chunk {idx + 1} ---")
        print(f"Name: {chunk['name']}")
        print(f"Department: {chunk['department']}")
        print(f"Email: {chunk['email']}")
        print(f"Courses: {chunk.get('courses_taught', [])}")
        print(f"Content:\n{chunk['content'][:500]}...")  # Limit output

    # === Build context ===
    context = truncate(build_context(top_chunks))
    include_memory = not name_path
    prompt = build_memory_prompt(memory, context, query,include_memory=include_memory)
    print("\n🧩 include_memory_in_prompt = {include_memory} and 📤 Final Prompt Sent to Phi-2:")
    print(prompt)

    # === Generate with Phi ===
    inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
    outputs = phi_model.generate(**inputs, max_new_tokens=300)
    raw_output = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = raw_output.split("Answer:")[-1].strip()

    # === Update memory and return
    memory = update_memory(memory, query, answer)
    return history + [
        {"role": "user", "content": query},
        {"role": "assistant", "content": answer}
    ], memory


# === Pipeline 2: Exam QA Pipeline ===
def exam_qa_pipeline(query, history=[]):
    print(f"\n🧠 Incoming Exam QA Query: {query}")

    prompt = f"""You are a university teaching assistant helping students prepare for exams.

{query}

Generate exactly 10 multiple choice questions with answers.
Each question should be clearly numbered 1 to 10 and followed by:
A. ...
B. ...
C. ...
D. ...
Answer: <Correct option letter>

Begin below:
"""

    inputs = phi_tokenizer(prompt, return_tensors="pt").to(phi_model.device)
    outputs = phi_model.generate(**inputs, max_new_tokens=700)
    response = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.split("Begin below:")[-1].strip()

    return history + [
        {"role": "user", "content": query},
        {"role": "assistant", "content": response}
    ]


# === Router ===
def gradio_chatbot(query, mode, history=[], memory=[]):
    print(f"\n🚦 Mode Selected: {mode}")
    if mode == "Exam QA Mode":
        updated_history = exam_qa_pipeline(query, history)
        return updated_history, memory
    else:
        updated_history, updated_memory = faculty_pipeline(query, history, memory)
        return updated_history, updated_memory


# === Gradio UI ===
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI Teaching Assistant\nChoose a mode and ask your question")

    mode_dropdown = gr.Dropdown(choices=["Faculty Research Mode", "Exam QA Mode"], value="Faculty Research Mode", label="Mode")
    chatbox = gr.Chatbot(type="messages", label="Chat History")
  # ✅ using messages format
    query_input = gr.Textbox(label="Ask your question...")
    clear = gr.Button("Clear Chat")

    memory_state = gr.State([])   # memory of last 5 QAs
    history_state = gr.State([])  # full chatbot history

    # with gr.Accordion("🔁 Memory", open=False):
    #     memory_box = gr.Textbox(label="Conversation Memory", lines=6)
    # query_input.submit(lambda query, memory: "\n\n".join([f"Q: {q}\nA: {a}" for q, a in memory]), inputs=[query_input, memory_state], outputs=memory_box)

    query_input.submit(gradio_chatbot, inputs=[query_input, mode_dropdown, history_state, memory_state], outputs=[chatbox, memory_state])
    query_input.submit(lambda _: "", inputs=query_input, outputs=query_input)  # clear textbox after submit
    clear.click(lambda: ([], []), outputs=[chatbox, memory_state])

demo.launch(debug=True, share=True)



/tmp/ipython-input-2235342107.py:215: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbox = gr.Chatbot(type="messages", label="Chat History")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9532e08ccd2b7cffa8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🚦 Mode Selected: Faculty Research Mode

🧠 Incoming Faculty Query: who is ayesha nir
🎯 Using name-based retrieval: Dr. Ayesha Nair (skip FAISS)

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Ayesha Nair
Department: Computer Science
Email: anair@aum.edu
Courses: ['Data Structures', 'Theory of Computation']
Content:
Name: Dr. Ayesha Nair
Designation: Associate Professor
Department: Computer Science
Email: anair@aum.edu
Courses Taught: Data Structures, Theory of Computation
Recent Publications: Understanding Heap Optimization (2023), On DFA Minimization (2022)
Ongoing Projects: AI-Driven Learning Systems
Research Interests: Automata Theory, Learning Algorithms
Research Keywords: Automata, Optimization, Learning Systems

🧭 Using FAISS fallback

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Ayesha Nair
Department: Computer Science
Email: anair@aum.edu
Courses: ['Data Structures', 'Theory of Computation']
Content:
Name: Dr. Ayesha Nair
Designation: Associate Professor
Depar

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


🚦 Mode Selected: Faculty Research Mode

🧠 Incoming Faculty Query: tell me about him
🔁 Ambiguity resolved via memory → using: Dr. Marcus Lee
🎯 Using name-based retrieval: Dr. Marcus Lee (skip FAISS)

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: Assistant Professor
Department: Information Systems
Email: mlee@aum.edu
Courses Taught: Database Management, Enterprise Systems
Recent Publications: NoSQL Trends in 2023
Ongoing Projects: Smart Campus Database Architecture
Research Interests: Data Modeling, Cloud Storage
Research Keywords: Databases, Cloud, Data Architecture

🧭 Using FAISS fallback

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: Assistant Profes

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🚦 Mode Selected: Faculty Research Mode

🧠 Incoming Faculty Query: provide his email
🔁 Ambiguity resolved via memory → using: Dr. Marcus Lee
🎯 Using name-based retrieval: Dr. Marcus Lee (skip FAISS)

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: Assistant Professor
Department: Information Systems
Email: mlee@aum.edu
Courses Taught: Database Management, Enterprise Systems
Recent Publications: NoSQL Trends in 2023
Ongoing Projects: Smart Campus Database Architecture
Research Interests: Data Modeling, Cloud Storage
Research Keywords: Databases, Cloud, Data Architecture

🧭 Using FAISS fallback

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: Assistant Profes

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🚦 Mode Selected: Faculty Research Mode

🧠 Incoming Faculty Query: what are his research interests
🔁 Ambiguity resolved via memory → using: Dr. Marcus Lee
🎯 Using name-based retrieval: Dr. Marcus Lee (skip FAISS)

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: Assistant Professor
Department: Information Systems
Email: mlee@aum.edu
Courses Taught: Database Management, Enterprise Systems
Recent Publications: NoSQL Trends in 2023
Ongoing Projects: Smart Campus Database Architecture
Research Interests: Data Modeling, Cloud Storage
Research Keywords: Databases, Cloud, Data Architecture

🧭 Using FAISS fallback

📥 Retrieved Context Chunks:

--- Chunk 1 ---
Name: Dr. Marcus Lee
Department: Information Systems
Email: mlee@aum.edu
Courses: ['Database Management', 'Enterprise Systems']
Content:
Name: Dr. Marcus Lee
Designation: As